In [2]:
import Pkg
cd("../")
Pkg.activate(".")

In [3]:
using FKTcheb
using SymPy
using TimerOutputs
using LinearAlgebra
using LowRankApprox
using Printf
using Random
using Plots
using Distributions
using StaticArrays
using Polynomials
using SpecialFunctions
using Combinatorics
using LaTeXStrings

┌ Info: Precompiling FKTcheb [8f6a1ce6-3be6-48da-82a4-714bd5a66102]
└ @ Base loading.jl:1317
┌ Warning: Package FKTcheb does not have Plots in its dependencies:
│ - If you have FKTcheb checked out for development and have
│   added Plots as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with FKTcheb
└ Loading Plots into FKTcheb from project dependency, future warnings for FKTcheb are suppressed.


In [4]:
include("src/factor.jl")
include("src/util.jl")
include("src/gegenbauer.jl")
include("src/hyperspherical.jl")

hypospherical (generic function with 1 method)

In [5]:
r = Sym("r")
spread_param = 6
dct_n         = 100 # Iterations for discrete cosine transform
kern          = 1 / (1+r^2)
mat_kern(x,y) = 1 / (1+norm(x-y)^2)
lkern         = lambdify(kern)
to            = TimerOutput()


 ──────────────────────────────────────────────────────────────────
                           Time                   Allocations      
                   ──────────────────────   ───────────────────────
 Tot / % measured:      512ms / 0.00%           55.5MiB / 0.00%    

 Section   ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────
 ──────────────────────────────────────────────────────────────────

In [6]:
timing_results = Dict()


Dict{Any, Any}()

In [ ]:
for num_points in 100_000:225_000:1_000_000
    random_samp = randperm(num_points)[1:1000]
    for d in 3:4:15
        x_vecs = [randn(d) / spread_param for _ in 1:num_points]
        mn = maximum(norm.(x_vecs))
        for i in 1:length(x_vecs)
            x_vecs[i] /= mn
        end
        truth_mat  = mat_kern.(x_vecs[random_samp], permutedims(x_vecs[random_samp]))
        truemat_frobnorm = norm(truth_mat)
        for err_tol_pow in 2
            GC.gc()
            err_tol = (1e-3)*(2^err_tol_pow)
            best_deg = guess_fkt_err(lkern, x_vecs, dct_n, err_tol)
            cfg = fkt_config(best_deg, d, dct_n, to, err_tol)
            fkt_t = 0
            U_mat=0
            for k in 1:3
                fkt_t += @elapsed U_mat = degen_kern_harmonic(lkern, x_vecs, cfg)
            end
            fkt_t/=3
            V_mat = transpose(U_mat)
            fkt_rank = size(U_mat, 2)

            fkt_guess = U_mat[random_samp,:]*V_mat[:, random_samp]
            fkt_err = (norm(fkt_guess-truth_mat)
                            /truemat_frobnorm)
            timing_results[(num_points, d, err_tol)] = (fkt_t, fkt_rank, fkt_err)
        end
    end
end



In [ ]:
ns = collect(100_000:225_000:1_000_000)
all_times = []
err_tol_pow = 2
err_tol = (1e-3)*(2^err_tol_pow)
for d in 3:4:15
    times = []
    for n in ns
        push!(times, timing_results[(n, d, err_tol)][1])
        println(timing_results[(n, d, err_tol)])
    end
    push!(all_times, times)
end
p = plot(thickness_scaling = 1.25, xlabel="N", ylabel="Time")
for k in 1:length(all_times)
    p=plot!(ns, all_times[k], label=string("d=",4k-1),  xaxis=:log10, xlim=(8e4, 1.5e6),
        yaxis=:log10, ylim=(3e-2, 9e1), legend=:topleft, linewidth=3)
end
p=plot!(ns, 0.0000006ns, label=L"$$t\sim N$$", linewidth=3, linestyle=:dash) # dotted guidelines
p=plot!(ns, 0.00000000008ns.^2, label=L"$$t\sim N^2$$", linewidth=3, linestyle=:dash)
p

# one single dataset, different subsets into it. 

In [7]:
err_results = Dict()

Dict{Any, Any}()

In [ ]:
for num_points in 1_000_000
    random_samp = randperm(num_points)[1:1000]
    for d in 3:4:15
        x_vecs = [randn(d) / spread_param for _ in 1:num_points]
        mn = maximum(norm.(x_vecs))
        for i in 1:length(x_vecs)
            x_vecs[i] /= mn
        end
        truth_mat  = mat_kern.(x_vecs[random_samp], permutedims(x_vecs[random_samp]))
        truemat_frobnorm = norm(truth_mat)
        for err_tol_pow in 5:9
            if d > 3 && err_tol_pow < 5 continue end
            GC.gc()
            err_tol = (1e-5)*(2.0^err_tol_pow)
            best_deg = guess_fkt_err(lkern, x_vecs, dct_n, err_tol)
            println(best_deg)
            cfg = fkt_config(best_deg, d, dct_n, to, err_tol)
            fkt_t = 0
            fkt_err = 0
            fkt_rank = 0
            mv_time = 0
            for k in 1:3
                fkt_t += @elapsed U_mat, diag_mat = degen_kern_harmonic(lkern, x_vecs, cfg)
                fkt_rank += size(U_mat, 2)
                sampmat = U_mat[random_samp,:]
                fkt_guess = sampmat*diagm(diag_mat)*transpose(sampmat)
                fkt_err += (norm(fkt_guess-truth_mat)
                                /truemat_frobnorm)
                mv_time += @elapsed res = U_mat*(diagm(diag_mat)*(transpose(U_mat)*rand(size(U_mat, 1))))
            end
            fkt_t/=3
            fkt_rank /= 3
            fkt_err /= 3
            mv_time /=3
            err_results[(num_points, d, err_tol)] = (fkt_t, fkt_rank, fkt_err, mv_time)
        end
    end
end



Using interval [0,1.9996754182730523]
16
8 orders needed
8 orders needed
8 orders needed
Using interval [0,1.9996754182730523]
16
8 orders needed
8 orders needed
8 orders needed
Using interval [0,1.9996754182730523]
14
7 orders needed
7 orders needed
7 orders needed
Using interval [0,1.9996754182730523]
12
6 orders needed
6 orders needed
6 orders needed
Using interval [0,1.9996754182730523]
10
5 orders needed
5 orders needed
5 orders needed
Using interval [0,2.0004497181644405]
16
8

In [ ]:
num_points = 1_000_000
d_to_nystrom_times = Dict()
d_to_nystrom_ranks = Dict()
d_to_nystrom_errs = Dict()
d_to_nystrom_mv_times = Dict()
for d in 3:4:15
    nystrom_times = []
    nystrom_ranks = []
    nystrom_errs  = []
    nystrom_mv_times = []

    random_samp = randperm(num_points)[1:1000]
    x_vecs = [randn(d) / spread_param for _ in 1:num_points]
    mn = maximum(norm.(x_vecs))
    for i in 1:length(x_vecs)
        x_vecs[i] /= mn
    end    
    truth_mat  = mat_kern.(x_vecs[random_samp], permutedims(x_vecs[random_samp]))
    truemat_frobnorm = norm(truth_mat)
    for idrnk in 1:20:120
        nystrom_t = 0
        nystrom_err = 0
        mv_time = 0
        for k in 1:3
            q_set = randperm(length(x_vecs))[1:idrnk]
            GC.gc()
            nystrom_t += @elapsed begin
                Nq =  mat_kern.(x_vecs, permutedims(x_vecs[q_set]))
                qmat = lu( mat_kern.(x_vecs[q_set], permutedims(x_vecs[q_set])))
            end
            nystrom_guess = Nq[random_samp,:] * (qmat \ transpose(Nq[random_samp,:]))
            nystrom_err += norm(nystrom_guess-truth_mat)/truemat_frobnorm
            mv_time += @elapsed res = (Nq 
                * (qmat \ 
                    (transpose(Nq)*rand(size(Nq, 1)))))
                

        end
        nystrom_err/=3
        nystrom_t/=3
        mv_time /= 3

        # Get error for Nystrom

        push!(nystrom_ranks, idrnk)
        push!(nystrom_errs, nystrom_err)
        push!(nystrom_times, nystrom_t)
        push!(nystrom_mv_times, mv_time)
        println(nystrom_errs[end])
        # println("Nystrom r=",idrnk, ", err=", nystromerror)
    end
    d_to_nystrom_times[d] = nystrom_times
    d_to_nystrom_errs[d] = nystrom_errs
    d_to_nystrom_ranks[d] = nystrom_ranks
    d_to_nystrom_mv_times[d] = nystrom_mv_times
end

In [ ]:
n = 1_000_000
d = 3

times = []
accs = []
ranks = []

fkt_mv_times = []
 for err_tol_pow in 0:9
    err_tol = (1e-5)*(2.0^err_tol_pow)
    push!(times, err_results[(n, d, err_tol)][1])
    push!(accs, err_results[(n, d, err_tol)][3])
    push!(ranks, err_results[(n, d, err_tol)][2])
    push!(fkt_mv_times, err_results[(n, d, err_tol)][4])
    println(err_results[(n, d, err_tol)])
end
p = plot(title="d=3, n=1,000,000")

p = plot!(accs, times, xlabel="Rel acc", ylabel="Time", label="FKT", 
     xlim=(2e-5,1e-1), legend=:topright,thickness_scaling = 1.25, xflip=false, xaxis=:log10, yaxis=:log10)
p = plot!(accs, fkt_mv_times, label="FKT mv")
p = plot!(d_to_nystrom_errs[d], d_to_nystrom_times[d], label="Nystrom")

p = plot!(d_to_nystrom_errs[d], d_to_nystrom_mv_times[d], label="Nystrom mv")

# add matvec, same error as factor, use 2-norm for factor

In [ ]:
n = 1_000_000
all_ys = []
for d in 3:4:15
    ys = []
    for err_tol_pow in 5:9
        err_tol = (1e-5)*(2.0^err_tol_pow)
   
        push!(ys, err_results[(n, d,err_tol)][3])
        println(err_results[(n, d,err_tol)][3])
    end
    push!(all_ys, ys)
end
p = plot(title=string("n=",n), thickness_scaling = 1.25, xlabel="Input err", ylabel="Result err", 
    xaxis=:log10, yaxis=:log10, legend=:bottomright)
for k in 1:length(all_ys)
    p=plot!([ (1e-5)*(2.0^err_tol_pow) for err_tol_pow in 5:9], all_ys[k], label=string("d=",4k-1))
end
p

#appendix

In [ ]:
n = 1_000_000
all_times = []
all_errs = []
for d in 3:4:15
    times = []
    errs = []

    for err_tol_pow in 5:9
        err_tol = (1e-5)*(2.0^err_tol_pow)      
        push!(times, err_results[(n, d,err_tol)][1])
        push!(errs, err_results[(n, d,err_tol)][3])
        println( err_results[(n, d,err_tol)])
    end
    push!(all_times, times)
    push!(all_errs, errs)
end
p = plot(title=string("n=",n), thickness_scaling = 1.25, ylabel="Time", xlabel="Result err", 
    xaxis=:log10, yaxis=:log10, legend=:topright)
for k in 1:length(all_times)
    p=plot!(all_errs[k], all_times[k], label=string("d=",4k-1), xflip=true, ylim =( 5e-1, 5e1))
end
p

In [ ]:
all_times = []
n = 1_000_000
d = 3

times = []
accs = []
ranks = []

for err_tol_pow in 0:9
    err_tol = (1e-5)*(2.0^err_tol_pow)
    push!(times, err_results[(n, d, err_tol)][1])
    push!(accs, err_results[(n, d, err_tol)][3])
    push!(ranks, err_results[(n, d, err_tol)][2])
end
p = plot(title="d=3, n=1,000,000")

p = plot!( ranks,accs, xlabel="Rank", ylabel="Rel acc", label="FKT", thickness_scaling = 1.25)
p = plot!(d_to_nystrom_ranks[d],d_to_nystrom_errs[d], yaxis=:log10, label="Nystrom", xlim=(0,40))
# Go further to the right FKT